In [1]:
import yaml
from utilz.utils import *
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from models.HDAAGT import *
from train_test import *
from utilz.utils import *
import datetime
import shutil

**First, we load Configuration file, then generate the data for plotting**

In [ ]:
cwd = os.getcwd()
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
with  open("/home/abdikhab/HDAAGT/configs/config.yaml") as file:
    config = yaml.safe_load(file)

ct = datetime.datetime.now().strftime("%m-%d-%H-%M") # Current time, used for saving the log
print(f"Total  # of GPUs {torch.cuda.device_count()}")
print(f"Using {device} device")

torch.cuda.empty_cache()
ZoneConf = Zoneconf(config['Zoneconf_path']) # This is the setting for zone configuration, you can find the image file at 'data/Zone Configuration.jpg'
# config non-indipendent parameters
config['NFeatures']= len(config["Columns_to_keep"]) # This goes to the data preparation process
config["input_size"] = config["NFeatures"] + 9 + 8 # This goes to the model, we add couple of more features to the input
config["output_size"] = len(config['xy_indx']) # The number of columns to predict
config['device'] = device
config['ct'] = ct
log_code = f"Learning rate: {config['learning_rate']}, Hidden size: {config['hidden_size']}, Batch size: {config['batch_size']}"
if config['verbal']:
    for arg in config.items(): # Print the arguments to the log file
        print(f"{arg[0]} = {arg[1]}", ct)
config['sos'] = torch.cat((torch.tensor(config['sos']) , torch.zeros(17)), dim=0).repeat(config['Nnodes'], 1).to(device)
config['eos'] = torch.cat((torch.tensor(config['eos']), torch.zeros(17)), dim=0).repeat(config['Nnodes'],1).to(device)

model = HDAAGT(config).to(device) # Here we load our model's class
print("Creating model from scratch")
criterion = nn.CrossEntropyLoss()
model.load_state_dict(torch.load(config['Load_Model_Path'], map_location=device))
learning_rate, hidden_size, num_layers, batch_size, ct= 1e-4, 128, 1, 128, '02-04-13-46'

code = f"LR{int(learning_rate*100000)}_HS{hidden_size}_NL{num_layers}_BS{batch_size} {ct}"
batch = 32
dataset = Scenes(config)
load = False
# load = True
if load:
    print("Loading CSV file ...", ct)
    Traffic_data = loadcsv(config['detection_path'], config['Headers'])
    dataset = Scene_Process_plotting(dataset, Traffic_data, config)
    dataset.save_class('Processed/video', cmnt = "video")
    del Traffic_data
else:
    dataset.load_class('Processed/video', cmnt = "video")


loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)



Total  # of GPUs 4
Using cuda:3 device
Creating model from scratch


/tmp/ipykernel_532248/2606044532.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(config['Load_Model_Path'], map_location=device))


**Then, we predict the results using the trained model weights**

In [3]:
Predicteddata = []
ActualTarget = []
ActualScene = []
model.eval()
for Scene, Target, Adj_Mat_Scene in loader:
    x, src_mask,adj_mat, target = prep_model_input(Scene, Adj_Mat_Scene,Target, config['sos'], config['eos'], config)
    Predicteddata += model(x, src_mask, adj_mat)
    ActualTarget += Target
    ActualScene += Scene

Predicteddata = torch.stack(Predicteddata)
ActualTarget = torch.stack(ActualTarget)
ActualScene = torch.stack(ActualScene)
a = Predicteddata[:,:].softmax(dim=-1)
top_values, top_indices = torch.topk(a, k = 3, dim=-1)
pred_tar0 = top_indices[:,:,:,:,0]
pred_tar = (top_indices*top_values).sum(-1)/top_values.sum(-1)
print(pred_tar0.shape, pred_tar.shape)
true_loss = F.mse_loss(top_indices[:,1:-1,:,:,0], ActualTarget[:,:,:,:2])
lvl2_loss = F.mse_loss(pred_tar0[:,1:-1], ActualTarget[:,:,:,:2])
print(true_loss, lvl2_loss)
error = torch.sqrt(torch.pow((pred_tar0[:,1:-1]- ActualTarget[:,:,:,:2]),2).sum(-1)).mean()
print(error)


torch.Size([1045, 17, 32, 2]) torch.Size([1045, 17, 32, 2])
tensor(25167.9941, device='cuda:3') tensor(25167.9941, device='cuda:3')
tensor(63.5890, device='cuda:3')


In [4]:
def moving_average_torch(x, window_size):
    """ x: Tensor of shape [batch_size, channels, time] """
    kernel = torch.ones(1, 1, window_size, device=x.device) / window_size
    # Use padding to maintain same output size
    x_padded = F.pad(x[1:-1].permute(1,2,0), (window_size - 1, 0), mode='replicate')  # causal padding
    xsmoothed = F.conv1d(x_padded[:,:1].to(torch.float), kernel)
    ysmoothed = F.conv1d(x_padded[:,1:].to(torch.float), kernel)
    return torch.cat((xsmoothed, ysmoothed), dim=1).permute(2,0,1)

**Now, we create a 2min videos of the scene along the whole dataset**

In [5]:

img0 = cv2.imread("data/no-car.JPG")
path = "/home/abdikhab/HDAAGT/Processed/resutls"
icon0 = cv2.imread("data/icon.png", cv2.IMREAD_UNCHANGED)  # Includes alpha
trf_light = cv2.imread("data/cropTrf.png", cv2.IMREAD_UNCHANGED)
init_pos = [45, 30, -60, 210, -60, 210, 120, 30, 120] # The initial position of the vehicles

crop_boxes = [[50,43, 282,293],[50,293,282,540],[50, 540, 282, 791], [50, 791, 282, 1043], [50, 1043, 282, 1295]] # Green, Red, Yellow, LeftGreen, LeftYellow
trfl_pos = [[911, 245], [384, 105], [254, 238], [750, 520]]
trfl_angle = [-60, 45, 135, -135]
def crop_traffic_light(trf_light, crop_boxes, resize):
    trf_light_imgs = []
    for boxes in crop_boxes:
        roi = trf_light[boxes[1]:boxes[3], boxes[0]:boxes[2]]
        roi = cv2.resize(roi, resize)
        trf_light_imgs.append(roi)
    return trf_light_imgs

def print_traffic_light(img, trf_light_imgs,trfl_pos, indices):
    for n, indx in enumerate(indices):
        indx = int(indx)-1
        M = cv2.getRotationMatrix2D((25,23), trfl_angle[n], 1.0)
        
        # Blend icon into background
        roi = img[trfl_pos[n][1]:trfl_pos[n][1]+46, trfl_pos[n][0]:trfl_pos[n][0]+50]
        icon = cv2.warpAffine(trf_light_imgs[indx], M, (50,46), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0,0))
        b, g, r, a = cv2.split(icon)
        overlay_color = cv2.merge((b, g, r))
        mask = cv2.merge((a, a, a)) / 255.0
        roi = roi * (1 - mask) + overlay_color * mask
        img[trfl_pos[n][1]:trfl_pos[n][1]+46, trfl_pos[n][0]:trfl_pos[n][0]+50] = roi.astype(np.uint8) # this should be fixed

if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)
h0 = 60

lower_color = np.array([5, 100, 100, 0])  # lower bound of blue in BGR
upper_color = np.array([55, 255, 255, 255])
lists = list(range(0, len(ActualScene)))
target_color = [(np.random.randint(0,254), np.random.randint(0,254), np.random.randint(0,254)) for i in range(32)]
trf_light_imgs = crop_traffic_light(trf_light, crop_boxes, (50, 46))

for jj in range(0, 5): # The 5 can be changed to len(ActualScene)//30):
    scene = ActualScene[jj]
    pred = moving_average_torch(pred_tar0[0,], 3)
    video_path = os.path.join(path, f"pred{jj}.mp4")
    video = cv2.VideoWriter(video_path,cv2.VideoWriter_fourcc(*'mp4v'),2,(1024,1024))
    img = img0.copy()
    for i in range (1,15): # Scene
        tmp_img = img.copy()
        for n in range(32):
            if scene[i,n,0] > h0 and scene[i-1,n,0] > h0:
                h = int(h0 * torch.exp(-scene[i,n,12]))
                # Rotation parameters
                center = (h // 2, h // 2)
                icsize = (2*(h//2), 2*(h//2))
                icon = cv2.resize(icon0, icsize)

                mask = cv2.inRange(icon, lower_color, upper_color)
                # Replace the old color with the new one using the mask
                icon[mask > 0, :3] = target_color[n]

                angle = 180*torch.atan2(-(scene[i,n,1]-scene[i-1,n,1]), (scene[i,n,0] - scene[i-1,n,0]))/torch.pi
                M = cv2.getRotationMatrix2D(center, int(angle), 1.0)
                rotated_icon = cv2.warpAffine(icon, M, icsize, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0,0))
                # Split rotated icon into BGR and alpha
                icon_rgb = rotated_icon[:, :, :3]
                alpha = rotated_icon[:, :, 3] / 255.0
                # Set position to place icon
                # Define ROI on the background
            
                roi = tmp_img[int(scene[i,n,1]-center[0]):int(scene[i,n,1]+center[0]), int(scene[i,n,0]-center[1]):int(scene[i,n,0]+center[1])]
                # Blend icon into background
                for c in range(3):
                    roi[:, :, c] = roi[:, :, c] * (1 - alpha) + icon_rgb[:, :, c] * alpha


                cv2.line(img, (int(scene[i,n,0]), int(scene[i,n,1])),(int(scene[i-1,n,0]), int(scene[i-1,n,1])), color = target_color[n], thickness= 4)
                
                cv2.putText(tmp_img, "Recording History", (412, 580), fontFace=cv2.FONT_HERSHEY_TRIPLEX, thickness= 3, fontScale=1, color=(0, 128, 0))
                # Replace the region in the original image
                tmp_img[int(scene[i,n,1]-center[0]):int(scene[i,n,1]+center[0]), int(scene[i,n,0]-center[1]):int(scene[i,n,0]+center[1])] = roi
        light_stat = torch.max(scene[i,:,3:7], dim=0)[0]
        print_traffic_light(tmp_img, trf_light_imgs,trfl_pos, light_stat)
        video.write(tmp_img)
    for m in lists[jj*30:jj*30+30]:
        tar = ActualTarget[m]
        pred = pred_tar0[m,] # moving_average_torch(pred_tar0[m,], 3)
        img = img0.copy()
        tetha0 = torch.zeros(32)
        for i in range (1,15): # Target
            tmp_img = img.copy()
            for n in range(32):
                R = torch.sqrt((pred[i,n,1]-512)**2 + (pred[i,n,0]-512)**2)/512
                if R < 0.85 and tar[i,n,1] > 50 and tar[i-1,n,1]> 50:
                    
                    h = int(h0 * torch.exp(-tar[i,n,12]))
                    center = (h // 2, h // 2)
                    icsize = (2*(h//2), 2*(h//2))
                    icon = cv2.resize(icon0, icsize)

                    mask = cv2.inRange(icon, lower_color, upper_color)
                    icon[mask > 0, :3] = target_color[n]

                    angle = 180*torch.atan2(-(scene[i,n,1]-scene[i-1,n,1]), (scene[i,n,0] - scene[i-1,n,0]))/torch.pi

                    M = cv2.getRotationMatrix2D(center, int(angle), 1.0)
                    rotated_icon = cv2.warpAffine(icon, M, icsize, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0,0))
                    # Split rotated icon into BGR and alpha
                    icon_rgb = rotated_icon[:, :, :3]
                    alpha = rotated_icon[:, :, 3] / 255.0
                    roi = tmp_img[int(tar[i,n,1]-center[0]):int(tar[i,n,1]+center[0]), int(tar[i,n,0]-center[1]):int(tar[i,n,0]+center[1])]

                    # Blend icon into background
                    for c in range(3):
                        roi[:, :, c] = roi[:, :, c] * (1 - alpha) + icon_rgb[:, :, c] * alpha

                    # Replace the region in the original image
                    cv2.line(img, (int(tar[i,n,0]), int(tar[i,n,1])),(int(tar[i-1,n,0]), int(tar[i-1,n,1])), color = target_color[n], thickness= 4)
                    cv2.circle(img, (int(pred[i, n, 0]), int(pred[i, n, 1])), 5, color = target_color[n], thickness= -1)
                    tmp_img[int(tar[i,n,1]-center[0]):int(tar[i,n,1]+center[0]), int(tar[i,n,0]-center[1]):int(tar[i,n,0]+center[1])] = roi
                    cv2.putText(tmp_img, "Predicting...", (412, 580),fontFace=cv2.FONT_HERSHEY_TRIPLEX ,thickness= 3, fontScale=1, color =(0, 128, 0) )
            light_stat = torch.max(tar[i,:,3:7], dim=0)[0]
            print_traffic_light(tmp_img, trf_light_imgs,trfl_pos, light_stat)      
            video.write(tmp_img)
    video.release()
    print("Videos created successfully")
cv2.waitKey(0)
cv2.destroyAllWindows()


Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully
Videos created successfully


*Model details*

In [5]:
from torchinfo import summary  # more flexible than torchsummary
learning_rate, hidden_size, num_layers, batch_size, ct= 1e-4, 128, 1, 128, '02-04-13-46'
code = f"LR{int(learning_rate*100000)}_HS{hidden_size}_NL{num_layers}_BS{batch_size} {ct}"
# Predicteddata  = torch.load(os.path.join(os.getcwd(),'Pickled', code + 'Predicteddata.pt'), weights_only=True)
ActualTarget = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Target.pt'), weights_only=True)
ActualScene = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Scene.pt'), weights_only=True)
Adj_Mat_Scene = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Adj_Mat_Scene.pt'), weights_only=True)
# Adj_Mat_Target = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Adj_Mat_Target.pt'), weights_only=True)
# test_loss = torch.load(os.path.join(os.getcwd(),'Pickled', code + 'test_losses.pt'), weights_only=True)
epoch_losses = torch.load(os.path.join(cwd,'Pickled', f'epoch_losses.pt'), weights_only=True)

# Dummy input example (adjust shapes accordingly)
x = attach_sos_eos(ActualScene[:1], config['sos'], config['eos'])
adj_mat = torch.cat((torch.ones_like(Adj_Mat_Scene[:1,:1]), Adj_Mat_Scene[:1], torch.ones_like(Adj_Mat_Scene[:1,:1])), dim=1)
src_mask = create_src_mask(x)
# custom_summary(model, x, src_mask, adj_mat)
summary(model, input_size=[x.size(), src_mask.size(), adj_mat.size()], device=device)

Layer (type:depth-idx)                        Output Shape              Param #
HDAAGT                                        [1, 17, 32, 2, 1024]      --
├─Encoder_DAAG: 1-1                           [32, 17, 384]             --
│    └─Positional_Encoding_Layer: 2-1         [32, 17, 384]             296,064
│    │    └─ModuleList: 3-1                   --                        262,144
│    │    └─DAAG_Layer: 3-2                   [1, 17, 32, 256]          211,456
│    │    └─TemporalConv: 3-3                 [32, 17, 128]             20,892
│    │    └─MultiheadAttention: 3-4           [32, 17, 384]             591,360
│    │    └─Dropout: 3-5                      [32, 17, 384]             --
│    └─Traffic_Encoding_Layer: 2-2            [32, 17, 384]             1,152
│    │    └─ModuleList: 3-6                   --                        1,248
│    │    └─LayerNorm: 3-7                    [1, 17, 32, 17]           34
│    │    └─Linear: 3-8                       [1, 17, 32, 128]   

*Model diagram illustration*

In [6]:
from torchviz import make_dot
model.eval()
output = model(x, src_mask, adj_mat)

# Create graph
dot = make_dot(output, params=dict(model.named_parameters()))
dot.format = 'png'
dot.render('models/HDAAGT_graph', cleanup=True)

'models/HDAAGT_graph.png'

***The rest is working but not maintained***

The 'ct' must be selected manually, it marks the unique model names, which contains date information.

In [ ]:
learning_rate, hidden_size, num_layers, batch_size, ct= 1e-4, 128, 1, 128, '02-04-13-46'
code = f"LR{int(learning_rate*100000)}_HS{hidden_size}_NL{num_layers}_BS{batch_size} {ct}"
Predicteddata  = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Predicteddata.pt'), weights_only=True)
ActualTarget = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Target.pt'), weights_only=True)
ActualScene = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Scene.pt'), weights_only=True)
Adj_Mat_Scene = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Adj_Mat_Scene.pt'), weights_only=True)
epoch_losses = torch.load(os.path.join(cwd,'Pickled', f'epoch_losses{ct``}.pt'), weights_only=True)


In [ ]:
rand_color = [
    (0, 0, 0),        # Black
    (0, 0, 255),      # Red
    (0, 255, 0),      # Green
    (255, 0, 0),      # Blue
    (0, 255, 255),    # Yellow
    (255, 255, 0),    # Cyan
    (255, 0, 255),    # Magenta
    (128, 0, 128),    # Purple
    (0, 165, 255),    # Orange
    (130, 0, 75),     # Indigo
    (128, 128, 0),    # Teal
    (0, 0, 128),      # Maroon
    (128, 0, 0),       # Navy
    (128, 128, 128),  # Gray
    (192, 192, 192),  # Silver
    (255, 255, 255),   # White
    (255, 165, 0),     # Brown
    (0, 128, 0),       # Olive
    (0, 128, 128),     # Aqua
    (128, 0, 128),     # Fuchsia
    (128, 128, 0),     # Lime
# Some more colors
    (50, 50, 50),      # Dark Gray
    (60, 120, 120),    # Sea Green
    (120, 30, 60),     # Crimson
    (100, 20, 120),    # Violet
    (10, 32, 64),       # Dark Blue
    (64,10, 32),        # Dark Red
    (10, 64, 32),       # Dark Green
    (16, 32, 64),       # Dark Aqua
    (32, 64, 16),       # Dark Lime
    (64, 32, 16),       # Dark Orange
    (64, 16, 32),       # Dark Pink

]

In [ ]:
import yaml
import re
from shapely.geometry import Point, Polygon
def Zoneconf1():
    ZoneConf = []
    with open('utilz/ZoneConf.yaml') as file:
        ZonesYML = yaml.load(file, Loader=yaml.FullLoader)
        #convert the string values to float
        for _, v in ZonesYML.items():
            lst = []
            for _, p  in v.items():    
                for x in p[0]:
                    b = re.split(r'[,()]',p[0][x])
                    lst.append((float(b[1]), float(b[2])))
            ZoneConf.append(lst)
    return ZoneConf

def zonefinder(BB, Zones):
    for i, zone in enumerate(Zones):
        Poly = Polygon(zone)
        if Poly.contains(Point(BB[0], BB[1])):
            return i
    return 100

In [ ]:
zoneconf = Zoneconf1()
zones = torch.empty(0,2)
zonesw0 = torch.empty(0,2)
err = 0
for m in range(440):
    tar = target[m,-1].to('cpu')
    pred = pred_tar[m,-4:-1].mean(0).to('cpu')
    for i in range(32):
        zt = zonefinder(tar[i],zoneconf)
        zp = zonefinder(pred[i], zoneconf)
        if zp == 100 and zt != 100:
            # print(pred[i], tar[i], zp, zt)
            err +=1
        else:
            zonesw0 = torch.cat((zonesw0,torch.tensor([[zp,zt]])), dim=0)
            if zt !=100:
                zones = torch.cat((zones,torch.tensor([[zp,zt]])), dim=0)

print(err)


In [ ]:
a = (zones[:,0]==zones[:,1]).sum()
b = zones.size(0)
print("Accuracy without zone 100 is ",a, b, a/b )
a = (zonesw0[:,0]==zonesw0[:,1]).sum()
b = zonesw0.size(0)
print("Accuracy with zone 100 is ",a, b, a/b )

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# Compute the confusion matrix
cm = confusion_matrix(zones[:,1], zones[:,0])

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm )
disp = disp.plot(cmap=plt.cm.YlGn,values_format='g')
# disp.plot()
plt.xlabel('Predicted Zones')
plt.ylabel('True Zones')
plt.title('Confusion Matrix')
SMALL_SIZE = 8
MEDIUM_SIZE = 12
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)
disp = disp.plot(cmap=plt.cm.YlGnBu,values_format='g')

In [ ]:
import matplotlib.pyplot as plt
import torch.nn.functional as F

batch = 32
Predicteddata = []
for n in range(0, ActualScene.size(0), batch):
    x, src_mask,adj_mat, target = prep_model_input(ActualScene[n:n+batch], Adj_Mat_Scene[n:n+batch], ActualTarget[n:n+batch], config['sos'], config['eos'], config)
    Predicteddata += model(x, src_mask, adj_mat)

Predicteddata = torch.stack(Predicteddata)
a = Predicteddata[:,:].softmax(dim=-1)
top_values, top_indices = torch.topk(a, k = 5, dim=-1)
pred_tar0 = top_indices[:,:,:,:,0]
pred_tar = (top_indices*top_values).sum(-1)/top_values.sum(-1)
print(pred_tar0.shape, pred_tar.shape)
true_loss = F.mse_loss(top_indices[:,1:-1,:,:,0], ActualTarget[:,:,:,1:3])
lvl2_loss = F.mse_loss(pred_tar0[:,1:-1], ActualTarget[:,:,:,1:3])
print(true_loss, lvl2_loss)
error = torch.sqrt(torch.pow((pred_tar0[:,1:-1]- ActualTarget[:,:,:,1:3]),2).sum(-1)).mean()
print(error)

In [ ]:

img0 = cv2.imread("/home/abdikhab/Traj_Pred/RawData/no-car.JPG")
path = "/home/abdikhab/New_Idea_Traj_Pred/results"
for m in range(440):
    img = img0.copy()
    scene = scene[m]
    tar = target[m,-1]
    pred = pred_tar0[m,-2]
    for i in range(32):
        
        egox, egoy = scene[:, i, 0], scene[:,i, 1]
        targetx, targety = tar[i, 0], tar[i, 1]
        # if dir != 0:
        for n in range(int(egox.size(0)-1)):
            if egox[n] > 10 and egox[n+1] > 10:
                cv2.line(img, (int(egox[n]), int(egoy[n])),(int(egox[n+1]), int(egoy[n+1])), color = rand_color[i], thickness= 4)
                # cv2.circle(img, (int(egox[n]), int(egoy[n])), 5, rand_color[i], -1)
        cv2.circle(img, (int(tar[i, 0]), int(tar[i, 1])), 10, rand_color[i], -1)
        cv2.rectangle(img, (int(pred[i, 0]), int(pred[i, 1])), (int(pred[i, 0]+15), int(pred[i, 1]+15)), rand_color[i], -1)
    
    cv2.imwrite(os.path.join(path, f"pred{m}.jpg"), img)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
print("Images created successfully")

In [ ]:
colors = {
'aliceblue':            '#F0F8FF',
'antiquewhite':         '#FAEBD7',
'aqua':                 '#00FFFF',
'aquamarine':           '#7FFFD4',
'azure':                '#F0FFFF',
'beige':                '#F5F5DC',
'bisque':               '#FFE4C4',
'black':                '#000000',
'blanchedalmond':       '#FFEBCD',
'blue':                 '#0000FF',
'blueviolet':           '#8A2BE2',
'brown':                '#A52A2A',
'burlywood':            '#DEB887',
'cadetblue':            '#5F9EA0',
'chartreuse':           '#7FFF00',
'chocolate':            '#D2691E',
'coral':                '#FF7F50',
'cornflowerblue':       '#6495ED',
'cornsilk':             '#FFF8DC',
'crimson':              '#DC143C',
'cyan':                 '#00FFFF',
'darkblue':             '#00008B',
'darkcyan':             '#008B8B',
'darkgoldenrod':        '#B8860B',
'darkgray':             '#A9A9A9',
'darkgreen':            '#006400',
'darkkhaki':            '#BDB76B',
'darkmagenta':          '#8B008B',
'darkolivegreen':       '#556B2F',
'darkorange':           '#FF8C00',
'darkorchid':           '#9932CC',
'darkred':              '#8B0000',
'darksalmon':           '#E9967A',
'darkseagreen':         '#8FBC8F',
'darkslateblue':        '#483D8B',
'darkslategray':        '#2F4F4F',
'darkturquoise':        '#00CED1',
'darkviolet':           '#9400D3',
'deeppink':             '#FF1493',
'deepskyblue':          '#00BFFF',
'dimgray':              '#696969',
'dodgerblue':           '#1E90FF',
'firebrick':            '#B22222',
'floralwhite':          '#FFFAF0',
'forestgreen':          '#228B22',
'fuchsia':              '#FF00FF',
'gainsboro':            '#DCDCDC',
'ghostwhite':           '#F8F8FF',
'gold':                 '#FFD700',
'goldenrod':            '#DAA520',
'gray':                 '#808080',
'green':                '#008000',
'greenyellow':          '#ADFF2F',
'honeydew':             '#F0FFF0',
'hotpink':              '#FF69B4',
'indianred':            '#CD5C5C',
'indigo':               '#4B0082',
'ivory':                '#FFFFF0',
'khaki':                '#F0E68C',
'lavender':             '#E6E6FA',
'lavenderblush':        '#FFF0F5',
'lawngreen':            '#7CFC00',
'lemonchiffon':         '#FFFACD',
'lightblue':            '#ADD8E6',
'lightcoral':           '#F08080',
'lightcyan':            '#E0FFFF',
'lightgoldenrodyellow': '#FAFAD2',
'lightgreen':           '#90EE90',
'lightgray':            '#D3D3D3',
'lightpink':            '#FFB6C1',
'lightsalmon':          '#FFA07A',
'lightseagreen':        '#20B2AA',
'lightskyblue':         '#87CEFA',
'lightslategray':       '#778899',
'lightsteelblue':       '#B0C4DE',
'lightyellow':          '#FFFFE0',
'lime':                 '#00FF00',
'limegreen':            '#32CD32',
'linen':                '#FAF0E6',
'magenta':              '#FF00FF',
'maroon':               '#800000',
'mediumaquamarine':     '#66CDAA',
'mediumblue':           '#0000CD',
'mediumorchid':         '#BA55D3',
'mediumpurple':         '#9370DB',
'mediumseagreen':       '#3CB371',
'mediumslateblue':      '#7B68EE',
'mediumspringgreen':    '#00FA9A',
'mediumturquoise':      '#48D1CC',
'mediumvioletred':      '#C71585',
'midnightblue':         '#191970',
'mintcream':            '#F5FFFA',
'mistyrose':            '#FFE4E1',
'moccasin':             '#FFE4B5',
'navajowhite':          '#FFDEAD',
'navy':                 '#000080',
'oldlace':              '#FDF5E6',
'olive':                '#808000',
'olivedrab':            '#6B8E23',
'orange':               '#FFA500',
'orangered':            '#FF4500',
'orchid':               '#DA70D6',
'palegoldenrod':        '#EEE8AA',
'palegreen':            '#98FB98',
'paleturquoise':        '#AFEEEE',
'palevioletred':        '#DB7093',
'papayawhip':           '#FFEFD5',
'peachpuff':            '#FFDAB9',
'peru':                 '#CD853F',
'pink':                 '#FFC0CB',
'plum':                 '#DDA0DD',
'powderblue':           '#B0E0E6',
'purple':               '#800080',
'red':                  '#FF0000',
'rosybrown':            '#BC8F8F',
'royalblue':            '#4169E1',
'saddlebrown':          '#8B4513',
'salmon':               '#FA8072',
'sandybrown':           '#FAA460',
'seagreen':             '#2E8B57',
'seashell':             '#FFF5EE',
'sienna':               '#A0522D',
'silver':               '#C0C0C0',
'skyblue':              '#87CEEB',
'slateblue':            '#6A5ACD',
'slategray':            '#708090',
'snow':                 '#FFFAFA',
'springgreen':          '#00FF7F',
'steelblue':            '#4682B4',
'tan':                  '#D2B48C',
'teal':                 '#008080',
'thistle':              '#D8BFD8',
'tomato':               '#FF6347',
'turquoise':            '#40E0D0',
'violet':               '#EE82EE',
'wheat':                '#F5DEB3',
'white':                '#FFFFFF',
'whitesmoke':           '#F5F5F5',
'yellow':               '#FFFF00',
'yellowgreen':          '#9ACD32'}